In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [32]:
INPUT_SIZE=28
HIDDEN_SIZE=500
NUM_CLASSES=10
NUM_EPOCHS=5
NUM_LAYERS=2
SEQUENCE_LENGTH=28
BATCH_SIZE=100
LEARNING_RATE=0.001

In [7]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=BATCH_SIZE, 
shuffle=False)

In [11]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES).to(device)

In [33]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(device)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [34]:
total_step = len(train_loader)
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, SEQUENCE_LENGTH, INPUT_SIZE).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(epoch+1, NUM_EPOCHS, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.3027
Epoch [1/5], Step [200/600], Loss: 2.2975
Epoch [1/5], Step [300/600], Loss: 2.3049
Epoch [1/5], Step [400/600], Loss: 2.2970
Epoch [1/5], Step [500/600], Loss: 2.3003
Epoch [1/5], Step [600/600], Loss: 2.3021
Epoch [2/5], Step [100/600], Loss: 2.2984
Epoch [2/5], Step [200/600], Loss: 2.3023
Epoch [2/5], Step [300/600], Loss: 2.2991
Epoch [2/5], Step [400/600], Loss: 2.3006
Epoch [2/5], Step [500/600], Loss: 2.3090
Epoch [2/5], Step [600/600], Loss: 2.3052


Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=L size=28x28 at 0x7F86B4CBF9E8>>
Traceback (most recent call last):
  File "/home/ayush/Projects/mlpython3.5/lib/python3.5/site-packages/PIL/Image.py", line 614, in __del__
    if hasattr(self, "_close__fp"):
KeyboardInterrupt


Epoch [3/5], Step [100/600], Loss: 2.3018
Epoch [3/5], Step [200/600], Loss: 2.3028
Epoch [3/5], Step [300/600], Loss: 2.3034
Epoch [3/5], Step [400/600], Loss: 2.3014
Epoch [3/5], Step [500/600], Loss: 2.3040
Epoch [3/5], Step [600/600], Loss: 2.3008
Epoch [4/5], Step [100/600], Loss: 2.3005
Epoch [4/5], Step [200/600], Loss: 2.3019
Epoch [4/5], Step [300/600], Loss: 2.3085
Epoch [4/5], Step [400/600], Loss: 2.3020
Epoch [4/5], Step [500/600], Loss: 2.2990
Epoch [4/5], Step [600/600], Loss: 2.3012
Epoch [5/5], Step [100/600], Loss: 2.3018
Epoch [5/5], Step [200/600], Loss: 2.3028
Epoch [5/5], Step [300/600], Loss: 2.3013
Epoch [5/5], Step [400/600], Loss: 2.2975
Epoch [5/5], Step [500/600], Loss: 2.3039
Epoch [5/5], Step [600/600], Loss: 2.3034


In [17]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 96.13 %


In [18]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')